Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
#train_dataset = train_dataset[0:500]
#train_labels = train_labels[0:500]
#train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
#logistic (gradient descent)
batch_size = 128
beta = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))#+tf.nn.l2_loss(weights)
  #l2_loss = tf.nn.l2_loss(weights)
  #loss = tf.reduce_mean(loss + beta*l2_loss)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

run logistic:

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.796739
Minibatch accuracy: 10.9%
Validation accuracy: 13.7%
Minibatch loss at step 500: 1.571822
Minibatch accuracy: 74.2%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 1.524115
Minibatch accuracy: 79.7%
Validation accuracy: 76.6%
Minibatch loss at step 1500: 0.798270
Minibatch accuracy: 83.6%
Validation accuracy: 77.7%
Minibatch loss at step 2000: 1.216756
Minibatch accuracy: 77.3%
Validation accuracy: 78.4%
Minibatch loss at step 2500: 1.242827
Minibatch accuracy: 74.2%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 0.645359
Minibatch accuracy: 82.0%
Validation accuracy: 78.5%
Test accuracy: 85.7%


neural network:

In [7]:
# init:
batch_size = 128
layer_size=1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #weights = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer_size]))
  hidden_biases = tf.Variable(tf.zeros([layer_size]))
  hidden=tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights)+hidden_biases)
  output_weights = tf.Variable(
    tf.truncated_normal([layer_size, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))  
  # Training computation.
  logits_relu = tf.matmul(hidden, output_weights) + output_biases
  
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits_relu, labels=tf_train_labels))
  l2_loss = tf.nn.l2_loss(hidden_weights)+tf.nn.l2_loss(output_weights)
  loss = tf.reduce_mean(loss + beta*l2_loss)  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_relu)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_logits = tf.matmul(valid_hidden, output_weights) + output_biases
  valid_prediction= tf.nn.softmax(valid_logits)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_logits = tf.matmul(test_hidden, output_weights) + output_biases
  test_prediction = tf.nn.softmax(test_logits)

run it:

In [8]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3490.913330
Minibatch accuracy: 12.5%
Validation accuracy: 36.3%
Minibatch loss at step 500: 21.357189
Minibatch accuracy: 78.9%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.895622
Minibatch accuracy: 83.6%
Validation accuracy: 84.5%
Minibatch loss at step 1500: 0.582260
Minibatch accuracy: 86.7%
Validation accuracy: 83.9%
Minibatch loss at step 2000: 0.741650
Minibatch accuracy: 82.8%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 0.857449
Minibatch accuracy: 79.7%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 0.586121
Minibatch accuracy: 88.3%
Validation accuracy: 83.3%
Minibatch loss at step 3500: 0.802265
Minibatch accuracy: 81.2%
Validation accuracy: 84.5%
Minibatch loss at step 4000: 0.769418
Minibatch accuracy: 80.5%
Validation accuracy: 84.8%
Minibatch loss at step 4500: 0.654623
Minibatch accuracy: 86.7%
Validation accuracy: 84.4%
Minibatch loss at step 5000: 0.703670
Minibatch accuracy: 83.6%
Validation acc

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 5001
train_dataset2 = train_dataset[0:500]
train_labels2 = train_labels[0:500]

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset2[offset:(offset + batch_size), :]
    batch_labels = train_labels2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3522.729736
Minibatch accuracy: 9.4%
Validation accuracy: 31.0%
Minibatch loss at step 500: 21.097851
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 1000: 0.493773
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 1500: 0.306319
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 0.290314
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 2500: 0.281194
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 0.276067
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 3500: 0.270045
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 4000: 0.267374
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 4500: 0.263815
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 5000: 0.263098
Minibatch accuracy: 100.0%
Valid

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
# init:
batch_size = 128
layer_size=1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #weights = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer_size]))
  hidden_biases = tf.Variable(tf.zeros([layer_size]))
  hidden=tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights)+hidden_biases)
  keep_prob = tf.placeholder("float")
  hidden_dropout=tf.nn.dropout(hidden,keep_prob)  
  output_weights = tf.Variable(
    tf.truncated_normal([layer_size, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))  
  # Training computation.
  logits_relu = tf.matmul(hidden_dropout, output_weights) + output_biases
  
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits_relu, labels=tf_train_labels))
  #l2_loss = tf.nn.l2_loss(hidden_weights)+tf.nn.l2_loss(output_weights)
  #loss = tf.reduce_mean(loss + beta*l2_loss)  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_relu)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_logits = tf.matmul(valid_hidden, output_weights) + output_biases
  valid_prediction= tf.nn.softmax(valid_logits)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_logits = tf.matmul(test_hidden, output_weights) + output_biases
  test_prediction = tf.nn.softmax(test_logits)

In [11]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 491.511475
Minibatch accuracy: 8.6%
Validation accuracy: 34.6%
Minibatch loss at step 500: 39.302856
Minibatch accuracy: 74.2%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 16.406914
Minibatch accuracy: 74.2%
Validation accuracy: 78.6%
Minibatch loss at step 1500: 23.614285
Minibatch accuracy: 76.6%
Validation accuracy: 80.1%
Minibatch loss at step 2000: 5.632072
Minibatch accuracy: 75.0%
Validation accuracy: 80.2%
Minibatch loss at step 2500: 7.894176
Minibatch accuracy: 63.3%
Validation accuracy: 80.1%
Minibatch loss at step 3000: 5.571589
Minibatch accuracy: 77.3%
Validation accuracy: 79.7%
Minibatch loss at step 3500: 6.928964
Minibatch accuracy: 71.9%
Validation accuracy: 79.2%
Minibatch loss at step 4000: 4.432501
Minibatch accuracy: 69.5%
Validation accuracy: 80.6%
Minibatch loss at step 4500: 2.185770
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 5000: 3.696079
Minibatch accuracy: 75.8%
Validation acc

In [12]:
#extreme overfitting
num_steps = 5001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset2[offset:(offset + batch_size), :]
    batch_labels = train_labels2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 464.064331
Minibatch accuracy: 13.3%
Validation accuracy: 30.6%
Minibatch loss at step 500: 0.594149
Minibatch accuracy: 99.2%
Validation accuracy: 77.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 3500: 0.401821
Minibatch accuracy: 99.2%
Validation accuracy: 78.3%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 5000: 0.000000
Minibatch accuracy: 100.0%
Validati

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [22]:
# init:
batch_size = 128
layer1_size=1024
#layer2_size=int(layer1_size*0.5)
layer2_size=layer1_size
layer3_size=int(layer2_size*0.5)
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #weights = tf.Variable(
  global_step = tf.Variable(0)  # count the number of steps taken.  
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1_size],stddev=2.0/batch_size))
  hidden_biases1 = tf.Variable(tf.zeros([layer1_size]))
  hidden1=tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights1)+hidden_biases1)
  keep_prob = tf.placeholder("float")
  hidden_dropout1=tf.nn.dropout(hidden1,keep_prob)
    
  hidden_weights2 = tf.Variable(
    tf.truncated_normal([layer1_size, layer2_size],stddev=2.0/layer1_size))
  hidden_biases2 = tf.Variable(tf.zeros([layer2_size]))
  hidden2=tf.nn.relu(tf.matmul(hidden_dropout1,hidden_weights2)+hidden_biases2)
  #keep_prob = tf.placeholder("float")
  hidden_dropout2=tf.nn.dropout(hidden2,keep_prob)

  hidden_weights3 = tf.Variable(
    tf.truncated_normal([layer2_size, layer3_size],stddev=2.0/layer2_size))
  hidden_biases3 = tf.Variable(tf.zeros([layer3_size]))
  hidden3=tf.nn.relu(tf.matmul(hidden_dropout2,hidden_weights3)+hidden_biases3)
  #keep_prob = tf.placeholder("float")
  hidden_dropout3=tf.nn.dropout(hidden3,keep_prob) 

  output_weights = tf.Variable(
    tf.truncated_normal([layer3_size, num_labels],stddev=2.0/layer3_size))
  output_biases = tf.Variable(tf.zeros([num_labels]))  
  # Training computation.
  logits_relu = tf.matmul(hidden_dropout3, output_weights) + output_biases
  
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits_relu, labels=tf_train_labels))
  l2_loss = tf.nn.l2_loss(hidden_weights1)+tf.nn.l2_loss(hidden_weights2)+tf.nn.l2_loss(hidden_weights3)+tf.nn.l2_loss(output_weights)
  loss = tf.reduce_mean(loss + beta*l2_loss)
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 3000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_relu)
  valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights1) + hidden_biases1)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, hidden_weights2) + hidden_biases2) 
  valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, hidden_weights3) + hidden_biases3)  
  valid_logits = tf.matmul(valid_hidden3, output_weights) + output_biases
  valid_prediction= tf.nn.softmax(valid_logits)
  test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights1) + hidden_biases1)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, hidden_weights2) + hidden_biases2)
  test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, hidden_weights3) + hidden_biases3)
  test_logits = tf.matmul(test_hidden3, output_weights) + output_biases
  test_prediction = tf.nn.softmax(test_logits)

In [23]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.380757
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.788127
Minibatch accuracy: 76.6%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 0.706641
Minibatch accuracy: 80.5%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 0.526202
Minibatch accuracy: 88.3%
Validation accuracy: 85.7%
Minibatch loss at step 2000: 0.604728
Minibatch accuracy: 85.9%
Validation accuracy: 86.5%
Minibatch loss at step 2500: 0.888443
Minibatch accuracy: 79.7%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 0.462668
Minibatch accuracy: 90.6%
Validation accuracy: 87.1%
Minibatch loss at step 3500: 0.768016
Minibatch accuracy: 80.5%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.656067
Minibatch accuracy: 85.9%
Validation accuracy: 87.6%
Minibatch loss at step 4500: 0.505621
Minibatch accuracy: 91.4%
Validation accuracy: 87.5%
Minibatch loss at step 5000: 0.667122
Minibatch accuracy: 86.7%
Validation accurac